In [1]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2

from fractions import Fraction

In [2]:
# replacement={
#     '83/83':'1',    
# }

# # Replacement
# multi[curation_col]= multi[curation_col].apply(
#     lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)

# matches=not_tube
# matches[curation_col]=matches['height'].astype(str)
# matches[curation_col]=matches[curation_col].apply(lambda x: f'"{x}"')
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'7\/165','.44',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))


# string = range_string
# lst = string.split('\n')
# builder_base = f'''
# def range_app(num):
# '''
# for val in lst:
#     if val.find(' - ') >-1:
#         original_val = val
#         lower, upper = val.split(' - ')
#         inner_f = f'''
#     if num >= {lower} and num <= {upper}:
#         return f'{lower}" - {upper}"{metric}'
#         '''
#         builder_base = builder_base + inner_f
#     if val.find('Greater than ') >-1:
#         original_val = val
#         upper = val.replace('Greater than ','')
#         inner_f = f'''
#     if num >= {upper}:
#         return f'{original_val}"{metric}'
#         '''
#         builder_base = builder_base + inner_f
# final_else = '''
#     else:
#         return r"n/a"
# '''
# exec(builder_base + final_else)
# print( builder_base + final_else)

# matches[curation_col]=matches['Min'].apply(range_app)
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
# matches


# # Find a list of values to iterate
# df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
# df_strategy
# list_values=df_strategy['value'].to_list()
# # list_values.append('coarse')

# kwp=KeywordProcessor()
# kwp.add_keywords_from_list(list_values)
# # Extract Key Words
# df['matches']=df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

In [3]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))  

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]  

def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))    

def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        print(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
#         print(num)
        return str(num)
    else:
        return numbers

# str(round(num, 2))
def remove_fraction(numbers):
    try:
        if numbers.find('/') > -1:   
            inte, fract = numbers.split(' ')
            nom, den = fract.split('/')
            tmp = float(nom)/float(den)
            num = int(inte) + tmp
#             print(num)
            return num
        else:
            return numbers
    except:
        return numbers

def remove_frac(numbers):
    if str(numbers).find('/') > -1:  
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
#         print(num)
        return tmp
    else:
        return numbers

def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()

# Definition to round a number
def round_string_float(x):
    try:
        return remove_exponent(Decimal(x).quantize(TWOPLACES))
    except:
        return x  

def frac2string(s):
    i, f = s.groups(0)
    f = round_string_float(Fraction(f))
    return str(int(i) + round_string_float(float(f)))

In [4]:
customer_id = '5'
customer_name='%bedbathandbeyond%'
buckets = "Fixings & Fasteners"
strategy_version_input=702
attribute_id_input=2674
formatted_attribute = 'weight'

In [5]:
# parameters
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [6]:
buckets = [x for x in enrich_db(f'''SELECT * FROM (SELECT * FROM strategy_buckets sb WHERE strategy_version_id = {strategy_version_input}) AS active_strat
INNER JOIN strategy_buckets_attributes sba ON active_strat.id = sba.strategy_bucket_id
WHERE sba.attribute_id = {attribute_id_input} 
AND sba.status = 'OPT_IN' ''')['bucket_id'].unique()] 

In [7]:
df = query_from_file(file_name='./query/uncurated.sql', params=params)
print(len(df)) 
df = df[df['bucket_id'].isin(buckets)]
height=df
len(df)

4672


2871

In [13]:
df[0:500]

product_id external_id  \
0     817376598    68468811   
1     817578954    41104583   
3     817412494    69498729   
5    1338869550    45423352   
8    1295407777    66918998   
21    988799881    62960229   
26   1012050787    42970972   
30    817681106    20403140   
33    817681101    20372484   
43   1347990108    16868655   
45   1347990797    17682040   
69   1350637331    64630175   
70   1350637328    68914813   
71   1350637327    68914974   
83   1350637105    40051062   
84   1319455770    20377892   
115  1296629253    12671881   
232  1319450477    69733026   
249  1350620414    68914806   
250  1350620415    68914820   
251  1149829719    69598960   
252  1127062111    69684911   
253  1131952101    69682862   
254  1357245747    69621457   
255  1357245746    69621442   
256   988764197    69621516   
257   988764068    69621475   
258  1174819857    69620805   
259   988469618    69562108   
260  1087325304    69435034   
261  1357221349    69305580   
262   817656068    69301155   
263   817656048    69295058   
264   817620537    20629397   
265   817380729    68695736   
270   817597780    20435288   
276  1357186495    67572830   
296   817344608    66874812   
298  1357174918    66784524   
299  1357172776    66545552   
300   817431855    66125778   
301  1167195457    66171713   
302   817572110    66001218   
305  1357155540    65430361   
306  1357155536    65430248   
307  1357155535    65430453   
308  1357155533    65430446   
309  1357155525    65430729   
310  1357155524    65430736   
311  1357155475    65430286   
312  1355839240    65430583   
313   988382801    65433652   
314  1357155438    65431726   
315  1357155274    65385760   
316  1357155363    65408841   
317  1357155359    65408261   
318  1357155352    65408926   
321  1357152976    65137338   
322   988641240    64888354   
323   988641482    64930893   
324  1357149102    64852089   
325  1355830729    64663395   
326   817213628    63820140   
331   817249390    60715920   
333  1347812568    41856369   
334   817465855    46645692   
335   817169596    45602726   
336   817178821    60264404   
337   817210780    62575256   
338   817492079    62575249   
339   817492085    62575201   
340   817210779    62575126   
341   817178104    60077936   
342   817177962    60056016   
343   817453732    14995510   
344   817160177    40558226   
345   988567274    20206345   
346  1311653896    69745134   
347  1203108670    69691877   
348  1176256710    69699420   
349  1231318522    69711482   
350  1071911392    20681197   
351  1199417538    69699576   
353  1165827876    69696492   
354  1227625958    69701683   
355  1227625957    69701682   
356   988566392    20703011   
357  1056089571    69646023   
358  1056089570    69646024   
359  1153048570    69646059   
360  1151868580    69686744   
361  1143067749    69676357   
362   988563310    69583483   
363   988563231    69582475   
364   988566034    69629362   
365  1269366978    69654300   
366   988564910    69605563   
368   817627787    69442650   
369   817403563    69275944   
370   988561794    69533131   
371   817638617    69533136   
372   988561642    69530158   
373   817635144    69510220   
374   817414347    69506426   
375   817410757    69491787   
376   817610434    68651749   
377   817684432    66643784   
378   817683895    66643890   
379   817361329    67636297   
380  1171819848    67373185   
381   988549973    69530156   
382   817686052    20572471   
383   817685835    20572457   
384   817688399    20572372   
385   817590666    67206605   
386   988546531    66471868   
387   817322008    65852156   
388   817342392    66803737   
389   817333829    66381518   
390   817315128    65524862   
391   817324943    61247321   
392   817571447    65937686   
393   988539718    63782288   
394   817286673    63782301   
395   988543303    65319512   
396   817310439    69495451   
397   817307435    65087015   
398   817307425    65086926   
399   

In [9]:
# regex_pattern_blank=r'(?i)(blanket)|()'
# df['matches_blanket']=df['long_desc'].apply(lambda x: re_extract(regex_pattern_blank, x))
# print(len(df[(df['matches_blanket'].astype(str)!='[]')])) 
# df=df[df['matches_blanket'].astype(str)!='[]']

regex_pattern=r'(?i)(.{20}lb.{20})|(.{20}(?<!light)weigh.{20})|()'
df['matches']=df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
print(len(df[(df['matches'].astype(str)=='[]')])) 

# df_top['matches']=df_top['matches']#.apply(lambda x: re.sub(r'[a-z]|[A-Z]|\:| ','',str(x)))#.apply(lambda x: re.sub(r"'\d+'",r'\d+ mm',str(x)))
# df_top['matches']=df_top['matches'].apply(lambda x: re.sub(r'[a-z]|[A-Z]|\:| ','',str(x)))
# df_top['matches']=df_top['matches'].apply(lambda x: re.sub(r'[a-z]|[A-Z]|\:| ','',str(x)))
df_na=df[df['matches'].astype(str)=='[]']
df_w=df[df['matches'].astype(str)!='[]']
df_na[curation_col]='n/a'
na_matches=df_na[['external_id',curation_col]]

166


In [10]:
na_matches

external_id Q:weight
3       69498729      n/a
21      62960229      n/a
69      64630175      n/a
70      68914813      n/a
71      68914974      n/a
249     68914806      n/a
250     68914820      n/a
254     69621457      n/a
255     69621442      n/a
257     69621475      n/a
301     66171713      n/a
302     66001218      n/a
305     65430361      n/a
306     65430248      n/a
307     65430453      n/a
308     65430446      n/a
309     65430729      n/a
310     65430736      n/a
311     65430286      n/a
313     65433652      n/a
314     65431726      n/a
315     65385760      n/a
316     65408841      n/a
317     65408261      n/a
318     65408926      n/a
324     64852089      n/a
391     61247321      n/a
422     69742559      n/a
423     69742560      n/a
424     69742568      n/a
427     69738357      n/a
428     69737481      n/a
429     69737479      n/a
430     69742563      n/a
432     69685934      n/a
434     69691007      n/a
447     69525113      n/a
463     65386620      n/a
500     20057565      n/a
501     14951091      n/a
625     44380311      n/a
699     44380328      n/a
721     67450176      n/a
749     46750426      n/a
1182    63239966      n/a
1215    64630168      n/a
1290    69689936      n/a
1293    69622476      n/a
1294    69487814      n/a
1299    20509019      n/a
1390    66698890      n/a
1395    64940441      n/a
1396    64662886      n/a
1400    62628426      n/a
1411    67059416      n/a
1414    66171799      n/a
1415    66171911      n/a
1419    65430231      n/a
1420    65430255      n/a
1421    65430514      n/a
1422    65433379      n/a
1423    65431849      n/a
1424    65408285      n/a
1427    64561202      n/a
1489    61247338      n/a
1498    69742569      n/a
1503    69742562      n/a
1504    69742565      n/a
1505    69742564      n/a
1506    64940786      n/a
1545    64851402      n/a
1566    60523082      n/a
1608    69095269      n/a
1644    16071110      n/a
1682    46750433      n/a
2078    69621500      n/a
2079    69621497      n/a
2080    69621495      n/a
2081    69621489      n/a
2082    69621484      n/a
2091    68958336      n/a
2103    65386705      n/a
2104    65385777      n/a
2115    17300865      n/a
2145    69601706      n/a
2151    66106845      n/a
2157    45583278      n/a
2237    69742171      n/a
2238    63533507      n/a
2239    44380335      n/a
2255    44927233      n/a
2270    20475727      n/a
2346    46750389      n/a
2390    45346262      n/a
2393    69738358      n/a
2394    69738361      n/a
2395    69738359      n/a
2396    69737480      n/a
2397    69737483      n/a
2411    43171866      n/a
2496    68529307      n/a
2497    65883556      n/a
2498    64662121      n/a
2499    68266066      n/a
2516    66171379      n/a
2525    65431443      n/a
2526    65385517      n/a
2533    61205802      n/a
2613    69742567      n/a
2614    69742558      n/a
2616    69742551      n/a
2617    69742552      n/a
2618    69742554      n/a
2682    45772931      n/a
2698    16048054      n/a
2821    17974300      n/a
3202    69742172      n/a
3203    69762347      n/a
3204    17974297      n/a
3207    69487810      n/a
3216    62960236      n/a
3229    68868383      n/a
3232    16071102      n/a
3233    16048070      n/a
3339    45543081      n/a
3342    69621493      n/a
3343    69621492      n/a
3373    66106371      n/a
3374    66171973      n/a
3382    65430118      n/a
3383    65430088      n/a
3384    65431269      n/a
3385    64851785      n/a
3431    69749924      n/a
3432    69738360      n/a
3433    69738356      n/a
3434    69737482      n/a
3437    40402185      n/a
3447    69742566      n/a
3448    69742561      n/a
3449    69742553      n/a
3468    65855096      n/a
3478    62807616      n/a
3488    69597760      n/a
3490    65257494      n/a
3491    65235249      n/a
3505    69722564      n/a
3608    46750440      n/a
3628    68868390      n/a
4068    69621483      n/a
4069    69621482      n/a
4072    68958541      n/a
4073    68955137      n/a
4

In [15]:
df['matches'].explode().value_counts()[0:500]

 you lounge in the MLB Flannel Plaid Pajam       36
ylish and relaxing MLB Flannel Plaid Pajam       36
onths fits up to 12 lb.; 6 - 9 months fits       33
fits children 24-33 lb. Size 2-4 Years fit       31
inforced extra-heavyweight nylon for seat and    25
m birth to 3 years; weighing 8-35 lb., and up    25
stributes a child's weight across back and sh    25
s children 31 to 34 lb. Size 4T fits child       24
children 25.5 to 29 lb. Size 2T fits child       24
cially licensed by MLB 1-year manufacturer       19
ures your favorite MLB team logo and color       19
toy train Features MLB team colors and log       19
poplin peached heavyweight Machine wash Impor    18
fits children 49-66 lb. Chlorine-resistanc       17
" L x 9\" W x 3\" H Weighs 5 oz. BANZ® Hearin    16
res with tiny LED bulbs to illuminate the        16
dB @ 1000Hz 40.53dB Weighs 6.7 oz BANZ® Heari    16
from birth up to 55 lb. Rear-facing infant       15
ize fits babies 6-9 lb. Size 0-6M runs a b       14
 foam fill A

In [11]:
regex_pattern=r'(?i)((?<!support.{0,10})(?<!Minimum.{0,10}?)(?<!Maxi?m?u?m?\s?c?h?i?l?d?.{0,10}?)(?<!accommodates.{10})(?<!light)weigh(?!t\s?capacity)(?!ing).{0,11}(?<!lb.{0,10})lb)|()'
df_w['matches']=df_w['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
print(len(df_w[(df_w['matches'].astype(str)!='[]')])) 
df_matches=df_w[(df_w['matches'].astype(str)!='[]')]
# df_matches['matches'].explode().values()[0:500] 
df_matches['matches'].explode().value_counts()[0:20]

335


weighs 9 lb           20
weighs 10 lb          16
weighs 7 lb           14
weighs 22 lb          13
weighs 5 lb           12
weighs 7.7 lb         11
weighs 7.6 lb         10
weighs 16 lb          10
weighs 9.5 lb         10
weighs 8 lb            8
weighs 28.6 lb         8
weighs 11 lb           8
weighs under 10 lb     8
weighs 6 lb            8
weighs 10.5 lb         8
weighs 9.9 lb          8
weighs 20 lb           7
weighs 14 lb           7
weight: 8 lb           7
weight: 9.6 lb         7
Name: matches, dtype: int64

In [11]:
# Weighs 2 lb. 45 lb
# Weighs 9 lb. 40 lb
# Weighs 9 lb. 60 lb
# weight: 5.5-25 lb
# \"
# df_matches['matches'].explode().value_counts()[1000:1500] 

In [12]:
df_matchez=df_matches
df_matchez['new_matches']=df_matchez['matches'].apply(lambda x: re.sub(r'lb\.|[a-z]|[A-Z]|:|\[|\]|​| |\200','',str(x))).apply(lambda x: round_string_float(x))
df_matchez['new_matches']=df_matchez['new_matches'].apply(lambda x: re.sub(r'1.65\"','1.65',str(x)))
# df_matchez['new_matches'].explode().value_counts()[500:1000]

In [13]:
def remove_slash(numbers):
    if numbers.find('\\') > -1:
        return ''
    else:
        return numbers

In [14]:
df_final=df_matchez
df_final['new_matches']=df_final['new_matches'].apply(lambda x: re.sub(r"'13','13'","'13'",str(x))).apply(lambda x: re.sub(r"\\200|'\\04\.5'|'\030.86'|","",str(x))).apply(lambda x: re.sub(r"\\","\\\\",str(x)))
df_final['new_matches']=df_final['new_matches'].apply(lambda x: remove_slash(x))
# df_final['new_matches'].explode().value_counts()[500:1000]

In [15]:
df_matchez['new_matches']=df_matchez['new_matches'].apply(lambda x: round_string_float(x))
# df_matchez['new_matches'].explode().value_counts()[0:500]

In [27]:
df_matchez['newest_matches']=df_matchez['new_matches'].apply(lambda x: re.sub(r"'(?!\d)"," lb'",str(x))).apply(lambda x: re.sub(r"(?<!lb)',"," lb',",str(x))).apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))   
regex_pattern_comma=r','
df_matchez['junk'] = df_matchez['newest_matches'].apply(lambda x: re_extract(regex_pattern_comma, str(x)))
matchez=df_matchez[df_matchez['junk'].astype(str)=='[]']
# matchez['newest_matches'].explode().value_counts()[0:500]

In [28]:
x='3 1/3'
remove_frac(remove_fraction(x))

3.3333333333333335

In [29]:
matchez['newest_matches']=matchez['newest_matches'].apply(lambda x: re.sub(r"'",'',str(x))).apply(lambda x: re.sub(r"lb\.59 lb",'0.59 lb',str(x))).apply(lambda x: re.sub(r"lb\.99 lb",'0.99 lb',str(x))).apply(lambda x: re.sub(r"lb\.5 lb",'0.5 lb',str(x)))
matchez['newest_matches']=matchez['newest_matches'].apply(lambda x: re.sub(r"\. lb",'lb',str(x))).apply(lambda x: re.sub(r"lb",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x))).apply(lambda x: re.sub(r"3 1\/3",'3.33',str(x)))

In [30]:
matchez['newest_matches']=matchez['newest_matches'].apply(lambda x: remove_frac(remove_fraction(x)))#.apply(lambda x: round_string_float(x))
matchez['newest_matches']=matchez['newest_matches'].apply(lambda x: str(x)+'lb')

In [31]:
matchez[0:500]

product_id external_id  \
61    1355844928    65884386   
103   1355704354    69741741   
168   1355784581    46309815   
236   1058099017    69644361   
237   1350637099    18819350   
238   1349058256    69763466   
239    817477818    62764636   
283   1350637087    17180827   
427   1350629134    69736315   
429   1294244876    69718481   
430   1295485517    69701735   
431   1124708240    69691333   
434   1196595820    69678022   
435   1196595799    69678016   
436   1196595775    69678009   
437   1196595753    69677883   
438   1196595968    69677959   
439   1196595966    69677957   
440   1196595938    69677954   
441   1196595924    69677948   
442   1196595904    69677939   
443   1196595888    69677932   
444    990729745    69662651   
445   1196595616    69677982   
446   1087662903    69638867   
447   1087662583    69638373   
450   1136006183    69629591   
452    988762738    65190715   
454    990722198    69602776   
455   1143271264    69582707   
456    988465770    69545011   
459   1329741479    64504483   
460   1329741475    64504438   
461   1329741472    64504322   
462    817402269    69335044   
468   1328427578    68738730   
469    817677861    68236250   
471    988414136    67714643   
474   1328409021    67621842   
475    988685796    61713789   
482    817648388    66875581   
485    817334062    66403944   
486    817332341    65051795   
487    817332339    65043776   
490   1349201343    66222033   
491    817324052    65954300   
493    988389849    65809556   
494    817430322    65844946   
495   1328380591    65644874   
496   1336377416    65644690   
497    817563854    65443057   
498   1329679169    65442548   
499   1337615828    65442531   
500    817313431    65442623   
501   1329679147    65438527   
502   1349186361    65051153   
515    817259054    61638440   
516   1329636999    60550262   
517   1329636996    60550200   
518   1329636836    60551191   
519   1329636813    60551559   
520   1329636804    60551672   
521   1329636219    60458964   
522   1329636784    60551856   
523   1329636207    60458841   
524   1329636190    60458667   
529    817184933    61737150   
530    817182173    61117808   
537    817170886    45987649   
538   1327018291    69746305   
539   1327018288    69746296   
540   1327018287    69746302   
541   1312872459    69744870   
542   1327018190    69742454   
549    817461329    44175412   
551    817165759    43886203   
552    817163521    42788638   
553   1338852569    69740809   
554   1320597040    69739477   
556    817457643    41328101   
559   1305275524    69735496   
560   1292867320    69735494   
561   1336303465    69735281   
563   1239494489    69715815   
566   1266557487    69712363   
572   1333852143    69722020   
573   1216106149    69722248   
574   1297639760    69721997   
575   1196389214    69695109   
577   1165827880    69696494   
578   1165827879    69696495   
579   1165827870    69696487   
581   1274975648    69687111   
582   1144346191    69687112   
583   1144346190    69687130   
584   1151868585    69686687   
587   1137017770    69676335   
588   1162658583    69676334   
589   1231318449    69709439   
590    990527874    69649361   
591   1173194943    69648882   
598   1067904397    69644465   
599   1058098354    69642180   
600    988566296    69642191   
601   1173195118    69658832   
603   1061932945    69654302   
604   1171835263    69652956   
605   1027881029    69641152   
607   1027881025    69641155   
609    988566210    69640301   
610    988566127    69637089   
611    988484920    69615505   
613    988564245    69598120   
615    988564018    69593796   
616    988564014    69593567   
617    988564013    69593566   
618    988563942    69592488   
619    988563813    69592273   
620    988563810    69592270   
621    988563797    69592281   
622    988563793    69592283   
624   1069921478    69630426   
627   1040067826    69627693   
628   1027880530    69627698   
629   

In [32]:
matchez[curation_col]=matchez['newest_matches']
final_matchez=matchez[['external_id',curation_col]]
print(len(final_matchez))
final_matchez[0:500]

4321


external_id   Q:weight
61      65884386       2 lb
103     69741741      48 lb
168     46309815     4.4 lb
236     69644361    31.9 lb
237     18819350   15.65 lb
238     69763466    1.46 lb
239     62764636      52 lb
283     17180827     5.5 lb
427     69736315    1.92 lb
429     69718481    80.5 lb
430     69701735     3.8 lb
431     69691333       3 lb
434     69678022      19 lb
435     69678016      19 lb
436     69678009      19 lb
437     69677883      19 lb
438     69677959      19 lb
439     69677957      19 lb
440     69677954      19 lb
441     69677948      19 lb
442     69677939      19 lb
443     69677932      19 lb
444     69662651     3.5 lb
445     69677982      19 lb
446     69638867    0.09 lb
447     69638373    0.09 lb
450     69629591      10 lb
452     65190715      12 lb
454     69602776     1.9 lb
455     69582707       8 lb
456     69545011     9.7 lb
459     64504483     2.2 lb
460     64504438     2.2 lb
461     64504322     2.2 lb
462     69335044     5.5 lb
468     68738730    1.67 lb
469     68236250    1.43 lb
471     67714643     1.6 lb
474     67621842       5 lb
475     61713789      38 lb
482     66875581    0.22 lb
485     66403944    0.36 lb
486     65051795    22.6 lb
487     65043776    22.2 lb
490     66222033      24 lb
491     65954300     7.5 lb
493     65809556  105.91 lb
494     65844946     0.4 lb
495     65644874       8 lb
496     65644690      31 lb
497     65443057       5 lb
498     65442548      28 lb
499     65442531      40 lb
500     65442623      24 lb
501     65438527      13 lb
502     65051153    2.25 lb
515     61638440   13.02 lb
516     60550262     3.7 lb
517     60550200     3.7 lb
518     60551191     3.7 lb
519     60551559     3.7 lb
520     60551672     3.7 lb
521     60458964     3.7 lb
522     60551856     3.7 lb
523     60458841     3.7 lb
524     60458667     3.7 lb
529     61737150      15 lb
530     61117808      15 lb
537     45987649    31.5 lb
538     69746305      75 lb
539     69746296      75 lb
540     69746302      75 lb
541     69744870      13 lb
542     69742454      75 lb
549     44175412     8.9 lb
551     43886203      15 lb
552     42788638    16.5 lb
553     69740809   43.86 lb
554     69739477    22.4 lb
556     41328101      18 lb
559     69735496      30 lb
560     69735494    29.7 lb
561     69735281      25 lb
563     69715815   24.25 lb
566     69712363    18.2 lb
572     69722020    1.87 lb
573     69722248     4.5 lb
574     69721997    2.27 lb
575     69695109   17.13 lb
577     69696494      14 lb
578     69696495      14 lb
579     69696487     6.8 lb
581     69687111    16.4 lb
582     69687112    16.4 lb
583     69687130    19.8 lb
584     69686687     1.6 lb
587     69676335    23.3 lb
588     69676334    23.3 lb
589     69709439     8.8 lb
590     69649361     0.4 lb
591     69648882     1.2 lb
598     69644465    6.25 lb
599     69642180    17.2 lb
600     69642191   14.72 lb
601     69658832     1.4 lb
603     69654302     6.8 lb
604     69652956     1.2 lb
605     69641152     8.6 lb
607     69641155     8.6 lb
609     69640301   10.42 lb
610     69637089     0.8 lb
611     69615505     6.7 lb
613     69598120    23.5 lb
615     69593796     2.3 lb
616     69593567       1 lb
617     69593566    1.32 lb
618     69592488     .12 lb
619     69592273    29.2 lb
620     69592270    29.2 lb
621     69592281    29.2 lb
622     69592283    29.2 lb
624     69630426    19.2 lb
627     69627693     7.5 lb
628     69627698   14.45 lb
629     69627697   14.45 lb
630     69628201    24.5 lb
631     69605863    12.5 lb
632     69605541    0.22 lb
635     69575661    17.6 lb
639     69604793    2.67 lb
640     69603746      10 lb
644     69544840   0.150 lb
649     69599110     3.3 lb
651     69536454    15.1 lb
652     69541171    13.5 lb
655     69582986       4 lb
656     69528171    20.1 lb
657     69536351   14.72 lb
658     69675198   14.72 lb
659     69518648    16.6 lb
663     69554455    13.6 lb
664     69509205       8 lb
6

In [25]:
# gc = gcloud.Gsheets()

In [26]:
# gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',wipe)

# send to the folder for upload

In [33]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/mult_buckets- {attribute}- upload_Shawn.csv',index=False)

In [34]:
looks_good('Bed Bath & Beyond', attribute, df, final_matchez)